In [2]:
import pandas as pd
import numpy as np

import os
import re
import nltk
#nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
#nltk.download()

import tensorflow as tf
import pickle
os.chdir('NLP_for_onedrive')
from NLP_function import * 

import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.preprocessing.text import text_to_word_sequence

os.chdir('Not_In_Database')

In [193]:
DF = pd.read_csv('unsupervised_learning/Training_data_final_ancenstral_HP_dense_cli_3.csv')

# aggreagate into list

df = DF.groupby('subclass', as_index=True)['merge'].aggregate(lambda x: list(x))
df = pd.DataFrame(df).reset_index()
df['merge'] = df['merge'].apply(lambda x: str(x))
df['class_train'] =  df['subclass'] + df['merge']

## Doc2Vec

### Model train

In [196]:
# Tokenization for Doc2Vec

X = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['class_train'])]
model = Doc2Vec(vector_size=150, alpha=0.025, min_alpha=0.00025, min_count=1, dm =2) 
# dm=1‘distributed memory’ (PV-DM), preserves the word order; dm =0 ‘distributed bag of words’ (PV-DBOW).
model.build_vocab(X)

labels = pd.DataFrame(X)


max_epochs = 100

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(X, total_examples=model.corpus_count, epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("M_Doc2Vec_BOW_25_hp_cli.model")
print("Model Saved")

iteration 0


C:\Users\Jingz\anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

### Prediction

In [197]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("M_Doc2Vec_BOW_25_hp_cli.model")

cli = pd.read_csv('Training_data_final_ancenstral_HP_dense_2.csv')

In [198]:
# remove class without information
cli = cli.fillna('无')
cli = cli[cli.y_cn!='无']
cli = cli[cli.y_cn!='正常']

cli = cli.drop_duplicates(subset='x_cn')
cli['x_cn'] = cli['x_cn'].apply(lambda x : change_number(x,'cn'))
cli = cli.reset_index(drop=True)
tem = pd.DataFrame({'pheno': cli['x_cn'], 'pheno_y': cli['HP_ancenstors']})

#### tokenization (top 3 most similar docs + top 5 terms search)

In [180]:
# choose top n terms to show
from translate import Translator
translator = Translator(from_lang='zh', to_lang='en')
topn = 10 

In [ ]:
### tokenization

i = random.randint(1, len(cli))
# i = 2016
print('row number is:', i)
txt = tem.iloc[i]

### find most similar doc using tags

txt['pheno_en_tok'] = txt['sim'] = txt['vec'] = txt['pheno_en'] = txt['pheno']
txt['pheno_en'] = translator.translate(txt['pheno'])
#txt['pheno_en_tok'] = text_to_word_sequence(txt['pheno_en'])
txt['vec']= word_tokenize(txt['pheno_en'].lower())
txt['vec'] = model.infer_vector(txt['vec'])
txt['sim'] = model.docvecs.most_similar([txt['vec']], topn = topn)
print('pheno: '+ txt['pheno']+ '\n' + 'pheno_en: '+ txt['pheno_en'] + '\n'+ 'real class: ' + txt['pheno_y'])

### predction class

from keras.preprocessing.text import text_to_word_sequence
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

### find vector of doc in hpo data using tags

for a in range(len(txt['sim'])):
        index = txt['sim'][a][0]
        print('prediction', a, '[', df.loc[int(index)]['subclass'],']')